### TensorBoard : TensorFlowの可視化フレームワーク

TensorBoardの主な目的は、訓練中にモデルの内部で起きていることすべてを視覚的に監視できるようにすること。<br>

<ul>
    <li>訓練中に指標を視覚的に監視</li>
    <li>モデルのアーキテクチャの監視</li>
    <li>活性化と勾配のヒストグラムの可視化</li>
    <li>埋め込みを３次元で調査</li>
</ul>

### IMDbの感情分析タスクでCNNを訓練

In [34]:
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):

    def set_model(self, model):
        self.model = model                                               
        layer_outputs = [layer.output for layer in model.layers]
        self.activations_model = keras.models.Model(model.input,
                                                    layer_outputs)       

    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')

        validation_sample = self.validation_data[0][0:1]                 
        activations = self.activations_model.predict(validation_sample)
        f = open('activations_at_epoch_' + str(epoch) + '.npz', 'w')     
        np.savez(f, activations)                                         
        f.close()      

In [35]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000 # 特徴量として考慮する単語の数
max_len = 500 # この数の単語を残してテキストをカット

(x_train, y_train), (x_test, y_test) =\
    imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len, name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()

model.compile(
    optimizer='rmsprop',
    loss = 'binary_crossentropy',
    metrics=['acc']
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_26 (Conv1D)           (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 98, 32)            0         
_________________________________________________________________
conv1d_27 (Conv1D)           (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_13 (Glo (None, 32)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# TensorFlowコールバックを使ってモデルを訓練
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir = 'my_log_dir', # ログファイルはこの場所に書き込まれる
        histogram_freq = 1, # 1エポックごとに活性化ヒストグラムを記録 
        embeddings_freq = 1 # 1エポックごとに埋め込みデータを記録
    )
]

history = model.fit(x_train, y_train,
                   epochs=20,
                    batch_size = 128,
                    validation_split=0.2,
                    callbacks=callbacks
                   )

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
19968/20000 [============================>.] - ETA: 0s - loss: 0.6369 - acc: 0.6523